# Support Vector Machine
* __SVM does not support RFECV__
* __We use Label encoding for target because SVM does not support multi valued target colum(list of lists) unlike other   classification algorithms.__

In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import RFECV

## Loading Dataset

In [ ]:
dataset = pd.read_csv("../../../Datasets/car_evaluation.txt", sep=",")
dataset.columns = ['buying','maintenance','doors','persons','lug_boot', 'safety', 'target']

In [ ]:
samples_count, features_count = dataset.shape
samples_count, features_count

## Separating target column from dataset

In [ ]:
target = dataset['target']
dataset = dataset.drop('target',axis=1)

## One-Hot Encoding of Dataset & Target column

### Dataset

In [ ]:
ohe_dataset = preprocessing.OneHotEncoder()
ohe_dataset.fit(dataset.values) 
dataset_encoded = ohe_dataset.transform(dataset.values).toarray()

### Target Column

In [ ]:
le_target = preprocessing.LabelEncoder()
target_encoded = le_target.fit_transform(target)

## Train & Test split

In [ ]:
x = dataset_encoded
y = target_encoded
train_x, test_x, train_y, test_y = train_test_split(x, y, random_state=0)

## Applying SVM

In [ ]:
svm = SVC()
svm.fit(train_x, train_y)

## Accuracy of SVM

In [ ]:
accuracy = svm.score(test_x,test_y)
print("%.2f" % accuracy)

# Apply SVM using Features Engineering (Automatic Features Selection)

## Univariant Statistics

In [ ]:
select = SelectPercentile(percentile=50)
select.fit(train_x, train_y)
train_x_selected = select.transform(train_x)
test_x_selected = select.transform(test_x)
svm_uni_fs = SVC(kernel='linear').fit(train_x_selected, train_y)
print("%.2f" % svm_uni_fs.score(test_x_selected, test_y))

## Model based features selection

In [ ]:
select.fit(train_x, train_y)
train_x_selected = select.transform(train_x)
test_x_selected = select.transform(test_x)
svm_model_fs = SVC(kernel='linear').fit(train_x_selected, train_y)
print("%.2f" % svm_model_fs.score(test_x_selected, test_y))

## Iterative feature selection

In [ ]:
select = RFE(SVC(kernel='linear'),n_features_to_select=100)
select.fit(train_x, train_y)
train_x_selected = select.transform(train_x)
test_x_selected = select.transform(test_x)
svm_ite_fs = SVC(kernel='linear').fit(train_x_selected, train_y)
print("%.2f" % svm_ite_fs.score(test_x_selected, test_y))

## Prediction by using input data from user

In [ ]:
buying = input("Enter buying category(vhigh,high,med,low)::")
maintenance = input("Enter buying maintenance(vhigh,high,med,low)::")
doors = input("Enter no of doors(2,3,4,5more)::")
persons = input("Enter no of persons(2,4,more)::")
lug_boot = input("Enter lug_boot category(small,med,big)::")
safety = input("Enter safety category(low,med,high)::")

In [ ]:
user_sample = [[buying, maintenance, doors, persons, lug_boot, safety]]
user_sample_encoded = ohe_dataset.transform(user_sample).toarray()
result_encoded = svm.predict(user_sample_encoded)
# result_encoded = svm_uni_fs.predict(user_sample_encoded)
# result_encoded = svm_model_fs.predict(user_sample_encoded)
# result_encoded = svm_ite_fs.predict(user_sample_encoded)
result_decoded = le_target.inverse_transform(result_encoded)
result = result_decoded[0]

In [ ]:
if result == 'unacc':
    print("Your car is unaccurate")
elif result == 'acc':
    print("Your car is accurate")
elif result == 'good':
    print("Your car is good")
else:
    print("Your car is very good")